In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=12,7

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4}
rev_mapping = {1: 'A', 2: 'B', 3: 'C', 4: 'D'}

In [4]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.model_selection import train_test_split

In [32]:

train_copy  = train.dropna().copy()
test_copy = test.copy()
train_copy['tr'] = 1
test_copy['tr'] = 0

appended = pd.concat([train_copy, test_copy], axis = 0)

cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
label_enc = {}
for col in cat_cols:
    appended[col] = appended[col].astype(str)
    enc = LabelEncoder().fit(appended[col])
    appended[col] = enc.transform(appended[col])
    label_enc[col] = enc
def id_features(data):
    df = data.copy()
    df['week'] = df['ID']%7
    df['month'] = df['ID']%30
    df['year'] = df['ID']%365
    df['quarter'] = df['ID']%90


    return df
appended = id_features(appended)
appended = pd.get_dummies(appended, columns = cat_cols)
train_copy = appended.loc[appended['tr'] == 1]
test_copy = appended.loc[appended['tr'] == 0]
Xcols = appended.drop(['Segmentation', 'tr'], axis = 1).columns   
train_copy = appended.loc[appended['tr'] == 1]
test_copy = appended.loc[appended['tr'] == 0]
Xcols = appended.drop(['Segmentation', 'tr'], axis = 1).columns
ycol = 'Segmentation'

X = train_copy[Xcols]
y = train_copy[ycol]

Xtest = test_copy[Xcols]

In [33]:
Xtest

,ID,Age,Work_Experience,Family_Size,week,month,year,quarter,Gender_0,Gender_1,...,Spending_Score_1,Spending_Score_2,Var_1_0,Var_1_1,Var_1_2,Var_1_3,Var_1_4,Var_1_5,Var_1_6,Var_1_7
0,458989,36,0.0,1.0,6,19,184,79,1,0,...,0,1,0,0,0,0,0,1,0,0
1,458994,37,8.0,4.0,4,24,189,84,0,1,...,0,0,0,0,0,0,0,1,0,0
2,458996,69,0.0,1.0,6,26,191,86,1,0,...,0,1,0,0,0,0,0,1,0,0
3,459000,59,11.0,2.0,3,0,195,0,0,1,...,1,0,0,0,0,0,0,1,0,0
4,459001,19,NaN,4.0,4,1,196,1,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,29,9.0,4.0,4,14,24,44,0,1,...,0,1,0,0,0,0,0,1,0,0
2623,467958,35,1.0,1.0,1,18,28,48,1,0,...,0,1,0,0,0,0,0,1,0,0
2624,467960,53,NaN,2.0,3,20,30,50,1,0,...,0,1,0,0,0,0,0,1,0,0
2625,467961,47,1.0,5.0,4,21,31,51,0,1,...,1,0,0,0,0,1,0,0,0,0


In [34]:
X.isnull().sum()

ID                  0
Age                 0
Work_Experience     0
Family_Size         0
week                0
month               0
year                0
quarter             0
Gender_0            0
Gender_1            0
Ever_Married_0      0
Ever_Married_1      0
Ever_Married_2      0
Graduated_0         0
Graduated_1         0
Graduated_2         0
Profession_0        0
Profession_1        0
Profession_2        0
Profession_3        0
Profession_4        0
Profession_5        0
Profession_6        0
Profession_7        0
Profession_8        0
Profession_9        0
Spending_Score_0    0
Spending_Score_1    0
Spending_Score_2    0
Var_1_0             0
Var_1_1             0
Var_1_2             0
Var_1_3             0
Var_1_4             0
Var_1_5             0
Var_1_6             0
Var_1_7             0
dtype: int64

In [35]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [36]:
from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [37]:
pipe = LGBMClassifier(n_estimators=300, max_features = .85, max_depth = 15, learning_rate = 1.1).fit(X, y)

In [48]:
pipe= make_pipeline(KNNImputer(n_neighbors = 10), LGBMClassifier(n_estimators=300, max_features = .85, max_depth = 15, learning_rate = 1.1)).fit(X, y)

In [49]:
preds=pipe.predict(Xtest)

In [50]:
preds

array(['B', 'C', 'D', ..., 'A', 'B', 'D'], dtype=object)

In [13]:
sub=pd.DataFrame({'ID':test['ID']})

In [51]:
sub['Segmentation']=preds

In [41]:
sub

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,D
3,459000,C
4,459001,D
...,...,...
2622,467954,D
2623,467958,A
2624,467960,A
2625,467961,B


In [52]:
sub.to_csv('Mysub13.csv',index=False)